In [ ]:
# 시각화 ( 대여소 현황 heatmap으로 지도 표기 )
# 대여-반납 이용횟수 라인 시각화


In [2]:
import mapboxgl
mapboxgl.__version__

'0.10.2'

In [ ]:
# 인증키 입력
token = '~인증키 입력란~'

In [3]:
# csv -> geojson 으로 만들기
import pandas as pd

df = pd.read_csv('final_final.csv', encoding='utf8', index_col=0)
df.head()

,대여시간,대여소번호,대여소명,대여구분코드,성별,연령대코드,이용건수,운동량,탄소량,이동거리(M),이용시간(분),주말,요일,대여소근처역,소재지,번호,위도,경도,동,산업종류
대여일자,,,,,,,,,,,,,,,,,,,,
2022-06-01,0,742,742. 등촌역 5번 출구 뒤,정기권,N,10대,1,48.30,0.46,1999.70,10,평일,수,등촌,양천구,742.0,37.550732,126.864578,목3동,사업시설관리 및 사업지원 서비스업
2022-06-01,0,2059,2059. 보라매공원 정문,정기권,N,10대,1,27.47,0.23,991.00,9,평일,수,보라매,동작구,2059.0,37.497078,126.917793,신대방2동,보건업 및 사회복지 서비스업
2022-06-01,0,1349,1349. 월계2교 버스정류장 앞,정기권,N,10대,1,20.63,0.20,868.17,3,평일,수,광운대,성북구,1349.0,37.623829,127.050201,번동,금융 및 보험업
2022-06-01,0,750,750. 연의근린공원 건너편,정기권,N,10대,1,13.97,0.13,560.00,4,평일,수,신정네거리,양천구,750.0,37.512157,126.835625,신정3동,운수업
2022-06-01,0,755,755. 목동1단지아파트 상가 앞,정기권,N,10대,1,12.15,0.14,590.00,3,평일,수,신목동,양천구,755.0,37.537869,126.881409,목5동,보건업 및 사회복지 서비스업


In [3]:
df['소재지'].value_counts()

강서구     392900
송파구     343992
영등포구    277652
노원구     215220
양천구     212326
마포구     197348
광진구     170003
강동구     168131
구로구     140953
성동구     135630
동대문구    132339
종로구     122615
서초구     117006
강남구     115378
중랑구     111542
관악구     104552
은평구     101873
성북구      99484
중구       85359
용산구      85056
서대문구     83501
동작구      82594
도봉구      81634
금천구      62651
강북구      58274
Name: 소재지, dtype: int64

In [ ]:
법정동코드
1. 종로구 11
2. 중구 14
3. 용산구 17
4. 성동구 20
5. 광진구 215
6. 동대문구 23
7. 중랑구 26
8. 성북구 29
9. 강북구 305
10. 도봉구 32
11. 노원구 35
12. 은평구 38
13. 서대문구 41
14. 마포구 44
15. 양천구 47
16. 강서구 50
17. 구로구 53
18. 금천구 545
19. 영등포구 56
20. 동작구 59
21. 관악구 62
22. 서초구 65
23. 강남구 68
24. 송파구 71
25. 강동구 74


In [25]:
from mapboxgl.utils import df_to_geojson
from tqdm import tqdm
list_gu = ['양천구', '동작구', '성북구', '종로구', '노원구', '강동구', '서초구', '강서구', '도봉구',
       '마포구', '용산구', '은평구', '광진구', '성동구', '동대문구', '구로구', '중랑구', '영등포구',
       '서대문구', '송파구', '강북구', '금천구', '강남구', '관악구', '중구']
#list_gu = ['도봉구']

for i in tqdm(list_gu):
    df2 = df[df['소재지']==i]
    df3 = df2

    #df3.drop_duplicates(['대여소번호'], inplace=True)
    li = df2['대여소번호'].value_counts()
    df3 = pd.DataFrame(li)
    df3.reset_index(inplace=True)
    df3.columns = ['대여소번호','대여횟수']
    df4 = pd.merge(df2, df3, on='대여소번호', how='left')
    geo_data = df_to_geojson(
        df=df4,
        lat='위도',
        lon='경도',
        properties=['대여소번호','대여횟수'],
        precision=5,
        filename = str(i)+'.geojson'
)

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:07<00:00,  7.90s/it]


In [26]:
from mapboxgl.utils import create_color_stops
from mapboxgl.viz import ClusteredCircleViz
import json

for i in tqdm(list_gu):
    geo_data = str(i)+'.geojson'

    with open(geo_data) as f:
        gdata = json.loads(f.read())


    token = token
    center = [126.981497, 37.518623]

    color_stops=create_color_stops([5,10,15,20,25,30], colors='BrBG')

    cviz = ClusteredCircleViz(
            access_token=token,
            data=gdata,
            color_stops = color_stops,
            radius_stops=[[1,5],[10,10],[50,15],[100,20]],
            radius_default=2,
            cluster_maxzoom=10,
            cluster_radius=30,
            label_size=12,
            opacity=0.9,
            center=center,
            zoom=3
    )
    cviz.create_html(str(i)+'.html')

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.11it/s]


In [16]:
# 전체 데이터

from mapboxgl.utils import df_to_geojson
from tqdm import tqdm


cn = df['대여소번호'].value_counts()
cn_df = pd.DataFrame(cn)
cn_df.reset_index(inplace=True)
cn_df.columns = ['대여소번호','대여횟수']
df_cn = pd.merge(cn_df, df, on='대여소번호', how='left')
df_cn.drop_duplicates(['대여소번호'], inplace=True)
geo_data = df_to_geojson(
    df=df_cn,
    lat='위도',
    lon='경도',
    properties=['대여소번호','대여횟수'],
    precision=5,
    filename = '서울시_Circle.geojson'
)

In [17]:
from mapboxgl.utils import create_color_stops
from mapboxgl.viz import CircleViz
import json

geo_data = '서울시_Circle.geojson'

with open(geo_data) as f:
    gdata = json.loads(f.read())


token = token
center = [126.981497, 37.518623]

color_breaks = [0,10,100,1000,10000]
color_stops = create_color_stops(color_breaks, colors='YlOrRd')


viz = CircleViz(
        access_token=token,
        data=gdata,
        height='400px',
        color_property='대여횟수',
        color_stops = color_stops,
        label_property='대여횟수',
        stroke_color = 'black',
        center=center,
        below_layer='waterway-label',
        zoom=3
)
viz.create_html('서울시_Circle.html')

In [18]:
cn = df['대여소번호'].value_counts()
cn_df = pd.DataFrame(cn)
cn_df.reset_index(inplace=True)
cn_df.columns = ['대여소번호','대여횟수']
df_cn = pd.merge(cn_df, df, on='대여소번호', how='left')
df_cn.drop_duplicates(['대여소번호'], inplace=True)
geo_data = df_to_geojson(
    df=df_cn,
    lat='위도',
    lon='경도',
    properties=['대여소번호','대여횟수'],
    precision=5,
    filename = '서울시_Cluster.geojson'
)

In [20]:
from mapboxgl.viz import ClusteredCircleViz

geo_data = '서울시_Cluster.geojson'

with open(geo_data) as f:
    gdata = json.loads(f.read())


token = token
center = [126.981497, 37.518623]

color_stops=create_color_stops([5,10,15,20,25,30], colors='BrBG')

cviz = ClusteredCircleViz(
        access_token=token,
        data=gdata,
        color_stops = color_stops,
        radius_stops=[[1,5],[10,10],[50,15],[100,20]],
        radius_default=2,
        cluster_maxzoom=10,
        cluster_radius=30,
        label_size=12,
        opacity=0.9,
        center=center,
        zoom=10.3
)
cviz.create_html('서울시_Cluster.html')

In [23]:
# 히트맵

from mapboxgl.utils import *
from mapboxgl.viz import *

geo_data = '서울시_Cluster.geojson'

with open(geo_data) as f:
    gdata = json.loads(f.read())

token = token
center = [126.981497, 37.518623]

heatmap_color_stops = create_color_stops([0.01,0.25,0.5,0.75,1], colors='RdPu')
heatmap_radius_stops = [[0,1], [15, 40]] #increase radius with zoom

color_breaks = [0,10,100,1000,10000]
color_stops = create_color_stops(color_breaks, colors='Spectral')

heatmap_weight_stops = create_weight_stops(color_breaks)

#Create a heatmap
hviz = HeatmapViz(data=gdata,
                 access_token=token,
                 weight_property='median_house_value',
                 weight_stops=heatmap_weight_stops,
                 color_stops=heatmap_color_stops,
                 radius_stops=heatmap_radius_stops,
                 opacity=0.9,
                 center=center,
                 zoom=3,
                 below_layer='waterway-label')

hviz.create_html('서울시_heatmap.html')